In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import sklearn
from sklearn import model_selection as ms
from sklearn.linear_model import LogisticRegression

# sns.set(rc={'figure.figsize':(11.7,8.27)})

import gzip

import copy

import import_ipynb
from _My_Functions import null_cols, rmse

importing Jupyter notebook from _My_Functions.ipynb


In [3]:
acc = pd.read_csv('../../../accepted_2007_to_2018Q4.csv.gz',
                  compression='gzip',
                  header=0,
                  sep=',',
                  quotechar='"')
acc

C:\Users\tdcho\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.00,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.00,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN

In [ ]:
# we will remove those rows
rows_to_drop_id = list(acc[acc['id'].str.contains('Total|Loans', na=False)].index)

acc = acc.drop(rows_to_drop_id)
acc

# Data Cleaning and Imputation

## Calculating ROI for comparison

In [ ]:
acc['roi'] = (acc['total_pymnt'] - acc['funded_amnt']) / acc['funded_amnt']

acc['roi']

## Getting rows that contain the desired target

In [ ]:
# keeping only the rows that have loan status of Paid Off, Default, or Charged Off
acc_train = copy.deepcopy(acc[(acc['loan_status'].str.contains('Paid|Default|Charged', na=False))])
acc_train = acc_train.reset_index()
acc_train = acc_train.drop('index', axis=1)
acc_train

In [ ]:
# create target label
target_train = copy.deepcopy(acc_train[['loan_status']])

target_train

In [ ]:
target_train[['loan_status']].value_counts()

In [ ]:
# converting loan statuses to binary (0, 1)
target_train['loan_status_binary'] = 1

# create mask for clarity on what is being changed
mask_fully = target_train['loan_status'].str.contains('Fully', na=False)

target_train.loc[mask_fully, 'loan_status_binary'] = 0

target_train = target_train.drop('loan_status', axis=1)

target_train

In [ ]:
target_train[['loan_status_binary']].value_counts()

In [ ]:
acc_train = acc_train.drop(['loan_status'], axis=1)

acc_train

## Splitting the dataset into train and prediction

In [ ]:
# getting indices of half of the samples
target_nodwnsmpl_index_0 = sorted(list(target_train\
                                       [target_train\
                                        ['loan_status_binary']==0].sample(frac=0.5,
                                                                          random_state=0).index))

target_nodwnsmpl_index_1 = sorted(list(target_train\
                                       [target_train\
                                        ['loan_status_binary']==1].sample(frac=0.5,
                                                                          random_state=0).index))

# combining the indices to create a list of indices containing 50% of both 0 and 1 
target_nodwnsmpl_index_0and1 = target_nodwnsmpl_index_0 + target_nodwnsmpl_index_1
target_nodwnsmpl_index_0and1

In [ ]:
len(target_nodwnsmpl_index_0and1)

In [ ]:
# getting the other half of the indices

target_nodwnsmpl_pred_index = sorted(list(set(list(range(0,
                                               target_train['loan_status_binary'].count()))) -\
                                set(target_nodwnsmpl_index_0and1)))

target_nodwnsmpl_pred_index

In [ ]:
len(target_nodwnsmpl_pred_index)

In [ ]:
# separating the dataframe into 2 dataframes: 1 for use/train and 1 for prediction comparison

acc_1b_train_downsample = acc_train.iloc[target_nodwnsmpl_index_0and1]
acc_1b_train_downsample = acc_1b_train_downsample.reset_index()
acc_1b_train_downsample = acc_1b_train_downsample.drop('index', axis=1)

target_1b_train_downsample = target_train.iloc[target_nodwnsmpl_index_0and1]
target_1b_train_downsample = target_1b_train_downsample.reset_index()
target_1b_train_downsample = target_1b_train_downsample.drop('index', axis=1)

# prediction set
acc_1b_pred_downsample = acc_train.iloc[target_nodwnsmpl_pred_index]
acc_1b_pred_downsample = acc_1b_pred_downsample.reset_index()
acc_1b_pred_downsample = acc_1b_pred_downsample.drop('index', axis=1)

target_1b_pred_downsample = target_train.iloc[target_nodwnsmpl_pred_index]
target_1b_pred_downsample = target_1b_pred_downsample.reset_index()
target_1b_pred_downsample = target_1b_pred_downsample.drop('index', axis=1)


acc_1b_pred_downsample

In [ ]:
target_1b_pred_downsample

In [ ]:
# drop unneeded columns

acc_pred = acc_1b_pred_downsample.drop(['collection_recovery_fee', 'desc', 'id', 'member_id',
                                        'next_pymnt_d', 'out_prncp', 'out_prncp_inv','pymnt_plan',
                                        'recoveries', 'title', 'total_pymnt_inv',
                                        'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp',
                                        'url', 'hardship_flag', 'hardship_type', 'hardship_reason',
                                        'hardship_status', 'deferral_term', 'hardship_amount',
                                        'hardship_start_date', 'hardship_end_date',
                                        'payment_plan_start_date', 'hardship_length',
                                        'hardship_dpd', 'hardship_loan_status',
                                        'orig_projected_additional_accrued_interest',
                                        'hardship_payoff_balance_amount',
                                        'hardship_last_payment_amount', 'disbursement_method',
                                        'debt_settlement_flag', 'debt_settlement_flag_date',
                                        'settlement_status', 'settlement_date', 'settlement_amount',
                                        'settlement_percentage', 'settlement_term', 'last_pymnt_d',
                                        'last_pymnt_amnt', 'last_fico_range_high',
                                          'last_fico_range_low', 'inq_last_12m', 'inq_last_6mths',
                                          'open_rv_24m', 'open_rv_12m', 'open_il_24m', 'open_il_12m',
                                          'open_acc_6m', 'num_tl_op_past_12m', 'num_tl_90g_dpd_24m',
                                          'num_tl_30dpd', 'num_tl_120dpd_2m', 'num_accts_ever_120_pd',
                                          'sec_app_chargeoff_within_12_mths',
                                          'sec_app_collections_12_mths_ex_med', 'zip_code',
                                          'addr_state'], axis=1)

acc_pred

In [ ]:
# acc_pred
              
# target_1b_pred_downsample

In [ ]:
# Running all of the imputation at once in 1 cell

# if employment length is 10+ years, insert 10 years
acc_pred['emp_length'] = np.where(acc_pred['emp_length']=='10+ years',
                                 '10 years', acc_pred['emp_length'])

# if employment length is 1 year , insert 1 years
acc_pred['emp_length'] = np.where(acc_pred['emp_length']=='1 year',
                                 '1 years', acc_pred['emp_length'])

# if employment length is <1, insert 0
acc_pred['emp_length'] = np.where(acc_pred['emp_length']=='< 1 year',
                                 '0 years', acc_pred['emp_length'])

# if employment length is null, insert -1
acc_pred['emp_length'] = np.where(acc_pred['emp_length'].isnull(),
                                 '-1 years', acc_pred['emp_length'])
# removing ' years' from the column
acc_pred['emp_length'] = acc_pred['emp_length'].str.replace(' years', '', regex=False)

# Convert to numeric type
acc_pred['emp_length'] = pd.to_numeric(acc_pred['emp_length'])

# Check if columns with dtype object should be ordinal
# Get columns with string values
cols_w_strings = acc_pred.loc[:, acc_pred.dtypes == object].columns

# Home Ownership
# Assigning values to home_ownership status
acc_pred.loc[acc_pred['home_ownership']=='OWN', 'home_ownership'] = 3
acc_pred.loc[acc_pred['home_ownership']=='MORTGAGE', 'home_ownership'] = 2
acc_pred.loc[acc_pred['home_ownership']=='ANY', 'home_ownership'] = 2
acc_pred.loc[acc_pred['home_ownership']=='OTHER', 'home_ownership'] = 1
acc_pred.loc[acc_pred['home_ownership']=='RENT', 'home_ownership'] = 0
acc_pred.loc[acc_pred['home_ownership']=='NONE', 'home_ownership'] = 0
acc_pred['home_ownership'] = acc_pred['home_ownership'].astype(int)

# Grade
# Grouping E, F, and G into F
acc_pred['grouped_grade_NO_EG'] = acc_pred['grade']
acc_pred['grouped_grade_NO_EG'] = acc_pred['grouped_grade_NO_EG'].str.replace('E', 'F', regex=False)
acc_pred['grouped_grade_NO_EG'] = acc_pred['grouped_grade_NO_EG'].str.replace('G', 'F', regex=False)

# replacing letter grades with numbers
# get sorted list of grades
grade_list = sorted(list(acc_pred['grade'].unique()))
i = 1
for grades in grade_list:
    acc_pred.loc[acc_pred['grade']==grades, 'grade'] = i
    i+=1
acc_pred['grade'] = acc_pred['grade'].astype(int)

# replacing letter grades with numbers for grouped grades
# get sorted list of grades
grade_list = sorted(list(acc_pred['grouped_grade_NO_EG'].unique()))
i = 1
for grades in grade_list:
    acc_pred.loc[acc_pred['grouped_grade_NO_EG']==grades, 'grouped_grade_NO_EG'] = i
    i+=1
acc_pred['grouped_grade_NO_EG'] = acc_pred['grouped_grade_NO_EG'].astype(int)

# Subgrade
# grouping subgrades 1, 2 vs 3, 4, 5
acc_pred['grouped_subgrade_12_345'] = acc_pred['sub_grade']
acc_pred['grouped_subgrade_12_345'] = acc_pred\
['grouped_subgrade_12_345'].str.replace('2','1',regex=False)
acc_pred['grouped_subgrade_12_345'] = acc_pred\
['grouped_subgrade_12_345'].str.replace('3','2', regex=False)
acc_pred['grouped_subgrade_12_345'] = acc_pred\
['grouped_subgrade_12_345'].str.replace('4','2', regex=False)
acc_pred['grouped_subgrade_12_345'] = acc_pred\
['grouped_subgrade_12_345'].str.replace('5','2', regex=False)

# grouping subgrades 1, 2, 3 vs 4, 5
acc_pred['grouped_subgrade_123_45'] = acc_pred['sub_grade']
acc_pred['grouped_subgrade_123_45'] = acc_pred\
['grouped_subgrade_123_45'].str.replace('2','1', regex=False)
acc_pred['grouped_subgrade_123_45'] = acc_pred\
['grouped_subgrade_123_45'].str.replace('3','1', regex=False)
acc_pred['grouped_subgrade_123_45'] = acc_pred\
['grouped_subgrade_123_45'].str.replace('4','2', regex=False)
acc_pred['grouped_subgrade_123_45'] = acc_pred\
['grouped_subgrade_123_45'].str.replace('5','2', regex=False)
acc_pred['grouped_subgrade_123_45'].value_counts()

# grouping Subgrades
acc_pred['grouped_subgrade'] = acc_pred['sub_grade']
acc_pred['grouped_subgrade'] = acc_pred['grouped_subgrade'].str.replace('E','F', regex=False)
acc_pred['grouped_subgrade'] = acc_pred['grouped_subgrade'].str.replace('G','F', regex=False)

# replacing subgrades with numbers
# get sorted list of subgrades
subgrade_list = sorted(list(acc_pred['sub_grade'].unique()))
i = 1
for subgrades in subgrade_list:
    acc_pred.loc[acc_pred['sub_grade']==subgrades, 'sub_grade'] = i
    i+=1
acc_pred['sub_grade'] = acc_pred['sub_grade'].astype(int)

# replacing subgrades with numbers
# get sorted list of subgrades
subgrade_list = sorted(list(acc_pred['grouped_subgrade_12_345'].unique()))
i = 1
for subgrades in subgrade_list:
    acc_pred.loc[acc_pred['grouped_subgrade_12_345']==subgrades, 'grouped_subgrade_12_345'] = i
    i+=1
acc_pred['grouped_subgrade_12_345'] = acc_pred['grouped_subgrade_12_345'].astype(int)

# replacing subgrades with numbers
# get sorted list of subgrades
subgrade_list = sorted(list(acc_pred['grouped_subgrade_123_45'].unique()))
i = 1
for subgrades in subgrade_list:
    acc_pred.loc[acc_pred['grouped_subgrade_123_45']==subgrades, 'grouped_subgrade_123_45'] = i
    i+=1
acc_pred['grouped_subgrade_123_45'] = acc_pred['grouped_subgrade_123_45'].astype(int)

# replacing subgrades with numbers
# get sorted list of subgrades
subgrade_list = sorted(list(acc_pred['grouped_subgrade'].unique()))
i = 1
for subgrades in subgrade_list:
    acc_pred.loc[acc_pred['grouped_subgrade']==subgrades, 'grouped_subgrade'] = i
    i+=1
acc_pred['grouped_subgrade'] = acc_pred['grouped_subgrade'].astype(int)

# Earliest Credit Line
# converting the column to datetime format
acc_pred['earliest_cr_line_dt'] = pd.DatetimeIndex(acc_pred['earliest_cr_line'])

# creating a new column that will show the number of months that credit has existed for customer
acc_pred['duration_of_credit_months'] = (pd.DatetimeIndex(acc_pred['issue_d']).year -\
                                        pd.DatetimeIndex(acc_pred['earliest_cr_line']).year)*12 +\
(pd.DatetimeIndex(acc_pred['issue_d']).month -\
 pd.DatetimeIndex(acc_pred['earliest_cr_line']).month)

# dropping the columns that helped create the new column
acc_pred = acc_pred.drop(['earliest_cr_line_dt', 'earliest_cr_line'], axis=1)

# Last Credit Pulled Date
# dropping the column bc it is used post-funding
acc_pred = acc_pred.drop('last_credit_pull_d', axis=1)

# Fico Score
acc_pred['fico_score'] = (acc_pred['fico_range_high'] + acc_pred['fico_range_low'])/2
acc_pred['sec_app_fico_score'] = (acc_pred['sec_app_fico_range_high'] +\
                                 acc_pred['sec_app_fico_range_low'])/2
acc_pred = acc_pred.drop(['fico_range_high', 'fico_range_low',
                        'sec_app_fico_range_high',
                        'sec_app_fico_range_low'], axis=1)
# Verification status
# if the income is verified, change to 1. If not verified, change to 0
acc_pred.loc[((acc_pred['verification_status']=='Source Verified') |\
             (acc_pred['verification_status']=='Verified')), 'verification_status'] = 1
acc_pred.loc[acc_pred['verification_status']=='Not Verified', 'verification_status'] = 0

# change dtype to int
acc_pred[['verification_status']] = acc_pred[['verification_status']].astype(int)

acc_pred.loc[acc_pred['application_type']=='Joint App', 'application_type'] = 2
acc_pred.loc[acc_pred['application_type']=='Individual', 'application_type'] = 1
acc_pred['application_type'] = acc_pred['application_type'].astype(int)

'''
for verification_status_joint where there are values in cols that start with
sec_app, but verification_status_joint is null
'''
# add in 'not verified' where applicable
acc_pred.loc[((acc_pred['verification_status_joint'].notnull()) &\
             (acc_pred['application_type']==1)), 'verification_status_joint'] = 'Not Verified'
acc_pred.loc[((acc_pred['verification_status_joint'].isnull()) &\
             (acc_pred['application_type']==2)), 'verification_status_joint'] = 'Not Verified'
acc_pred.loc[((acc_pred['verification_status_joint'].isnull()) &\
             (acc_pred['application_type']==1)), 'verification_status_joint'] = 'Individual'

# if the income is verified, change to 1. If not verified, change to 0
acc_pred.loc[((acc_pred['verification_status_joint']=='Source Verified') |\
                  (acc_pred['verification_status_joint']=='Verified')),
            'verification_status_joint'] = 1
acc_pred.loc[acc_pred['verification_status_joint']=='Individual',
            'verification_status_joint'] = -1
acc_pred.loc[acc_pred['verification_status_joint']=='Not Verified',
            'verification_status_joint'] = 0
acc_pred['verification_status_joint'] = acc_pred['verification_status_joint'].astype(int)

# Reducing Income if not Verified
acc_pred['annual_inc_reduced'] = acc_pred['annual_inc']
acc_pred.loc[acc_pred['verification_status']==0,
            'annual_inc_reduced'] = acc_pred['annual_inc'] * 0.7
acc_pred['annual_inc_joint_reduced'] = acc_pred['annual_inc_joint']
acc_pred.loc[acc_pred['verification_status_joint']==0,
            'annual_inc_joint_reduced'] = acc_pred['annual_inc_joint'] * 0.7

# Joint Earliest Credit Line
# converting column to datetime
acc_pred['sec_app_earliest_cr_line_dt'] = pd.DatetimeIndex(acc_pred['sec_app_earliest_cr_line'])

# creating a new column that will show the number of months
#that credit has existed for joint customer
acc_pred['sec_app_duration_of_credit_months']=(pd.DatetimeIndex\
                                              (acc_pred['issue_d']).year - pd.DatetimeIndex\
                                              (acc_pred['sec_app_earliest_cr_line']).year)*12 +\
(pd.DatetimeIndex(acc_pred['issue_d']).month -\
 pd.DatetimeIndex(acc_pred['sec_app_earliest_cr_line']).month)

# converting null values to -1
acc_pred.loc[acc_pred['sec_app_duration_of_credit_months'].isnull(),
            'sec_app_duration_of_credit_months'] = -1

acc_pred = acc_pred.drop(['sec_app_earliest_cr_line_dt', 'sec_app_earliest_cr_line',
                        'issue_d', 'sec_app_earliest_cr_line',
                        'sec_app_earliest_cr_line_dt'], axis=1)

cols_w_mths_since = [col for col in acc_pred.columns if col.startswith('mths_since_')]
for col in cols_w_mths_since:
    acc_pred.loc[acc_pred[col].isnull(), col] = 10000

# Get cols with null values
cols_w_null = []

# function that gives list of columns that contain null values
null_cols(acc_pred, cols_w_null)

# Fill in null values
# creates a list of the index where the column == 0
list(acc_pred[acc_pred['emp_length']==0].index)

# getting the columns where there are 0s
cols_w_0s_and_null = []
for col in cols_w_null:
    # if the list is not empty, then there are values == 0
    if list(acc_pred[acc_pred[col]==0].index)!=[]:
        cols_w_0s_and_null.append(col)
    for col_0 in cols_w_0s_and_null:
        acc_pred.loc[acc_pred[col_0].isnull(), col_0] = -1
    # rerun to get new cols w null values

cols_w_null = []
# function that gives list of columns that contain null values
null_cols(acc_pred, cols_w_null)

# Employment
acc_pred.loc[acc_pred['emp_title'].isnull(), 'emp_title'] = 'Unknown'

# Employment length
emplength_mean = acc_pred['emp_length'].mean()
acc_pred.loc[(acc_pred['emp_length'].isnull()) &\
            (acc_pred['emp_title'].notnull()), 'emp_length'] = emplength_mean
acc_pred.loc[(acc_pred['emp_length'].isnull()) &\
            (acc_pred['emp_title'].isnull()), 'emp_length'] = -1
acc_pred = acc_pred.drop(['emp_title'], axis=1)

# joint
# if no secondary app, convert to -1
acc_pred.loc[acc_pred['annual_inc_joint'].isnull(), 'annual_inc_joint'] = -1
acc_pred.loc[acc_pred['annual_inc_joint_reduced'].isnull(),'annual_inc_joint_reduced'] = -1
acc_pred.loc[acc_pred['sec_app_fico_score'].isnull(), 'sec_app_fico_score'] = -1

# others
acc_pred.loc[acc_pred['annual_inc'].isnull(), 'annual_inc'] = -1
acc_pred.loc[acc_pred['mo_sin_old_rev_tl_op'].isnull(), 'mo_sin_old_rev_tl_op'] = 10000
acc_pred.loc[acc_pred['duration_of_credit_months'].isnull(), 'duration_of_credit_months'] = -1
acc_pred.loc[acc_pred['sec_app_duration_of_credit_months'].isnull(),
            'sec_app_duration_of_credit_months'] = -1

acc_pred.loc[acc_pred['term'].str.contains('60', na=False), 'term'] = 1
acc_pred.loc[acc_pred['term'].str.contains('36', na=False), 'term'] = 0
acc_pred[['term']] = acc_pred[['term']].astype(int)

# total accounts
cols_w_acc = [col for col in acc_pred.columns if 'acc' in col]

# using the open_acc as a filler for the null values for total_acc
acc_pred.loc[acc_pred['total_acc'].isnull(), 'total_acc'] = acc_pred['open_acc']

cols_w_null = []
# function that gives list of columns that contain null values
null_cols(acc_pred, cols_w_null)
cols_w_null_list = list(acc_pred[cols_w_null].columns)

# getting the columns where there are 0s
for col in cols_w_null_list:
    acc_pred.loc[acc_pred[col].isnull(), col] = -1

cols_w_null = []
# function that gives list of columns that contain null values
null_cols(acc_pred, cols_w_null)

num_cols = [col for col in acc_pred.columns if '2' in col]
num_cols.remove('grouped_subgrade_12_345')
num_cols.remove('grouped_subgrade_123_45')
acc_pred = acc_pred.drop(num_cols, axis=1)

# Income : * Ratios
acc_pred[(acc_pred['application_type']==1) &\
        (acc_pred['annual_inc_joint']==acc_pred['annual_inc'])][['annual_inc_joint', 'annual_inc']]

# Giving weight to loan amount
# creating a new column
acc_pred['weighted_loan_amnt'] = acc_pred['loan_amnt']

# multiplying the new column by its perceived rank according to FICO score ranges
acc_pred.loc[acc_pred['fico_score']<=560, 'weighted_loan_amnt'] = acc_pred['weighted_loan_amnt'] * 5
acc_pred.loc[((acc_pred['fico_score']>560) & (acc_pred['fico_score']<=650)),
            'weighted_loan_amnt'] = acc_pred['weighted_loan_amnt'] * 4
acc_pred.loc[((acc_pred['fico_score']>650) & (acc_pred['fico_score']<=700)),
            'weighted_loan_amnt'] = acc_pred['weighted_loan_amnt'] * 3
acc_pred.loc[((acc_pred['fico_score']>700) & (acc_pred['fico_score']<=750)),
            'weighted_loan_amnt'] = acc_pred['weighted_loan_amnt'] * 2
acc_pred.loc[acc_pred['fico_score']>750, 'weighted_loan_amnt'] = acc_pred['weighted_loan_amnt'] * 1

# Giving weight to funded amount
# creating a new column
acc_pred['weighted_funded_amnt'] = acc_pred['funded_amnt']

# multiplying the new column by its perceived rank according to FICO score ranges
acc_pred.loc[acc_pred['fico_score']<=560,
            'weighted_funded_amnt'] = acc_pred['weighted_funded_amnt'] * 5
acc_pred.loc[((acc_pred['fico_score']>560) & (acc_pred['fico_score']<=650)),
            'weighted_funded_amnt'] = acc_pred['weighted_funded_amnt'] * 4
acc_pred.loc[((acc_pred['fico_score']>650) & (acc_pred['fico_score']<=700)),
            'weighted_funded_amnt'] = acc_pred['weighted_funded_amnt'] * 3
acc_pred.loc[((acc_pred['fico_score']>700) & (acc_pred['fico_score']<=750)),
            'weighted_funded_amnt'] = acc_pred['weighted_funded_amnt'] * 2
acc_pred.loc[acc_pred['fico_score']>750,
            'weighted_funded_amnt'] = acc_pred['weighted_funded_amnt'] * 1

# Giving weight to installment
# creating a new column
acc_pred['weighted_installment'] = acc_pred['installment']

# multiplying the new column by its perceived rank according to FICO score ranges
acc_pred.loc[acc_pred['fico_score']<=560,
            'weighted_installment'] = acc_pred['weighted_installment'] * 5
acc_pred.loc[((acc_pred['fico_score']>560) & (acc_pred['fico_score']<=650)),
            'weighted_installment'] = acc_pred['weighted_installment'] * 4
acc_pred.loc[((acc_pred['fico_score']>650) & (acc_pred['fico_score']<=700)),
            'weighted_installment'] = acc_pred['weighted_installment'] * 3
acc_pred.loc[((acc_pred['fico_score']>700) & (acc_pred['fico_score']<=750)),
            'weighted_installment'] = acc_pred['weighted_installment'] * 2
acc_pred.loc[acc_pred['fico_score']>750,
            'weighted_installment'] = acc_pred['weighted_installment'] * 1

# 1st borrower income
# Income to loan amount ratio
acc_pred['ratio_inc_loan'] = acc_pred['annual_inc'] / acc_pred['loan_amnt']
acc_pred['ratio_inc_weightedloan'] = acc_pred['annual_inc'] / acc_pred['weighted_loan_amnt']

# income reduced to loan amount ratio
acc_pred['ratio_redinc_loan'] = acc_pred['annual_inc_reduced'] / acc_pred['loan_amnt']
acc_pred['ratio_redinc_weightedloan'] = acc_pred['annual_inc_reduced']/acc_pred['weighted_loan_amnt']

# Income to funded amount ratio
acc_pred['ratio_inc_funded'] = acc_pred['annual_inc'] / acc_pred['funded_amnt']
acc_pred['ratio_inc_weightedfunded'] = acc_pred['annual_inc'] / acc_pred['weighted_funded_amnt']

# income reduced to funded amount ratio
acc_pred['ratio_redinc_funded'] = acc_pred['annual_inc_reduced'] / acc_pred['funded_amnt']
acc_pred['ratio_redinc_weightedfunded'] = acc_pred['annual_inc_reduced'] /\
acc_pred['weighted_funded_amnt']

# installment to income ratio
acc_pred['ratio_install_inc'] = acc_pred['annual_inc'] / acc_pred['installment']
acc_pred['ratio_weightedinstall_inc'] = acc_pred['annual_inc'] / acc_pred['weighted_installment']

# installment to reduced income ratio
acc_pred['ratio_install_redinc'] = acc_pred['annual_inc_reduced'] / acc_pred['installment']
acc_pred['ratio_weightedinstall_redinc'] = acc_pred['annual_inc_reduced'] /\
acc_pred['weighted_installment']

# joint borrower income
# Income to loan amount ratio
acc_pred['ratio_incjoint_loan'] = acc_pred['annual_inc_joint'] / acc_pred['loan_amnt']
acc_pred['ratio_incjoint_weightedloan'] = acc_pred['annual_inc_joint'] /\
acc_pred['weighted_loan_amnt']

# income reduced to loan amount ratio
acc_pred['ratio_redincjoint_loan'] = acc_pred['annual_inc_joint_reduced'] / acc_pred['loan_amnt']
acc_pred['ratio_redincjoint_weightedloan'] = acc_pred['annual_inc_joint_reduced'] /\
acc_pred['weighted_loan_amnt']

# Income to funded amount ratio
acc_pred['ratio_incjoint_funded'] = acc_pred['annual_inc_joint'] / acc_pred['funded_amnt']
acc_pred['ratio_incjoint_weightedfunded'] = acc_pred['annual_inc_joint'] /\
acc_pred['weighted_funded_amnt']

# income reduced to funded amount ratio
acc_pred['ratio_redincjoint_funded'] = acc_pred['annual_inc_joint_reduced'] / acc_pred['funded_amnt']
acc_pred['ratio_redincjoint_weightedfunded'] = acc_pred['annual_inc_joint_reduced'] /\
acc_pred['weighted_funded_amnt']

# Income to installment ratio
acc_pred['ratio_incjoint_installment'] = acc_pred['annual_inc_joint']/ acc_pred['installment']
acc_pred['ratio_incjoint_weightedinstallment'] = acc_pred['annual_inc_joint']/\
acc_pred['weighted_installment']

# Reduced income to installment ratio
acc_pred['ratio_redincjoint_installment'] = acc_pred['annual_inc_joint_reduced']/\
acc_pred['installment']
acc_pred['ratio_redincjoint_weightedinstallment'] = acc_pred['annual_inc_joint_reduced']/\
acc_pred['weighted_installment']

# Loan Amount vs Funded Amount
acc_pred['loan_minus_funded'] = acc_pred['loan_amnt'] - acc_pred['funded_amnt']
acc_pred['ratio_loan_funded'] = acc_pred['loan_amnt'] / acc_pred['funded_amnt']

# Dummification

In [ ]:
# getting a list of cols w dtype object

cols_w_obj = list(acc_pred.select_dtypes(include=['object']).columns)

cols_w_obj

In [ ]:
# dummification

acc_dum = pd.get_dummies(acc_pred, drop_first=True)

acc_dum

In [ ]:
list_pred = list(acc_dum.columns)

In [ ]:
list_train = ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment',
              'grade', 'sub_grade', 'emp_length', 'home_ownership', 'annual_inc',
              'verification_status', 'dti', 'mths_since_last_delinq', 'mths_since_last_record',
              'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'total_pymnt',
              'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint',
              'dti_joint', 'verification_status_joint', 'acc_now_delinq', 'tot_coll_amt',
              'tot_cur_bal', 'open_act_il', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
              'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'avg_cur_bal',
              'bc_open_to_buy', 'bc_util', 'delinq_amnt', 'mo_sin_old_il_acct',
              'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc',
              'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq',
              'mths_since_recent_revol_delinq', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats',
              'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0',
              'num_sats', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'pub_rec_bankruptcies',
              'tax_liens', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
              'total_il_high_credit_limit', 'revol_bal_joint', 'sec_app_inq_last_6mths',
              'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_open_act_il',
              'sec_app_num_rev_accts', 'sec_app_mths_since_last_major_derog', 'grouped_grade_NO_EG',
              'grouped_subgrade_12_345', 'grouped_subgrade_123_45', 'grouped_subgrade',
              'duration_of_credit_months', 'fico_score', 'sec_app_fico_score', 'annual_inc_reduced',
              'annual_inc_joint_reduced','sec_app_duration_of_credit_months', 'weighted_loan_amnt',
              'weighted_funded_amnt', 'weighted_installment', 'ratio_inc_loan',
              'ratio_inc_weightedloan', 'ratio_redinc_loan', 'ratio_redinc_weightedloan',
              'ratio_inc_funded', 'ratio_inc_weightedfunded', 'ratio_redinc_funded',
              'ratio_redinc_weightedfunded', 'ratio_install_inc', 'ratio_weightedinstall_inc',
              'ratio_install_redinc', 'ratio_weightedinstall_redinc', 'ratio_incjoint_loan',
              'ratio_incjoint_weightedloan', 'ratio_redincjoint_loan',
              'ratio_redincjoint_weightedloan', 'ratio_incjoint_funded',
              'ratio_incjoint_weightedfunded', 'ratio_redincjoint_funded',
              'ratio_redincjoint_weightedfunded', 'ratio_incjoint_installment',
              'ratio_incjoint_weightedinstallment', 'ratio_redincjoint_installment',
              'ratio_redincjoint_weightedinstallment', 'loan_minus_funded', 'ratio_loan_funded',
              'purpose_credit_card', 'purpose_debt_consolidation', 'purpose_educational',
              'purpose_home_improvement', 'purpose_house', 'purpose_major_purchase',
              'purpose_medical', 'purpose_moving', 'purpose_other', 'purpose_renewable_energy',
              'purpose_small_business', 'purpose_vacation', 'purpose_wedding',
              'initial_list_status_w', 'roi']

list_train.sort()
list_pred.sort()

if list_train == list_pred:
    print('identical')

# Scaling the dataset

In [ ]:
# we are preserving total_pymnt to use as our 

acc_totalpayment = copy.deepcopy(acc_dum[['total_pymnt', 'funded_amnt', 'roi']])

acc_dum = acc_dum.drop(['total_pymnt', 'funded_amnt', 'roi'], axis=1)

In [ ]:
# creating copies

import copy

acc_full = copy.deepcopy(acc_dum)

In [ ]:
# full scaling

from sklearn.preprocessing import MinMaxScaler

acc_dum_cols = list(acc_dum.columns)

for col in acc_dum_cols:
    acc_full[col] = MinMaxScaler().fit_transform(np.array(acc_full[col]).reshape(-1,1))
    
acc_full

In [ ]:
col_names = list(acc_full.columns)
to_add = ['total_pymnt', 'funded_amnt', 'roi']

for col in to_add:
    col_names.append(col)

acc_full = pd.DataFrame(np.concatenate((acc_full, acc_totalpayment), axis=1),
                             columns=col_names)
acc_full

In [ ]:
target_1b_pred_downsample

In [ ]:
acc_red = copy.deepcopy(acc_full[['annual_inc', 'fico_score', 'term',
                                  'loan_amnt', 'home_ownership', 'int_rate',
                                  'all_util', 'dti', 'funded_amnt', 
                                  'total_pymnt', 'roi']])

acc_red

In [ ]:
acc_red.to_csv("acc_vred_pred.csv.gz", index=False, compression="gzip")

In [ ]:
acc_full.to_csv("acc_2_pred_final.csv.gz", index=False, compression="gzip")

target_1b_pred_downsample.to_csv("target_2_pred_final.csv.gz",
                                  index=False,
                                  compression="gzip")

In [ ]:
acc_comparison = copy.deepcopy(acc_full[['funded_amnt', 'total_pymnt', 'roi']])

acc_comparison = pd.DataFrame(np.concatenate((acc_comparison, target_1b_pred_downsample), axis=1),
                              columns = ['funded_amnt', 'total_pymnt', 'roi', 'loan_status_binary'])

acc_comparison

In [ ]:
acc_comparison.to_csv("acc_2_pred_comparison.csv.gz",
                      index=False,
                      compression="gzip")